In [1]:
# added by PWL to force images to be downloaded before test starts
import glob
from IPython.display import HTML
png_list = glob.glob("*.png")
image_list = ""
for png in png_list:
    image_list+=f"<image src='{png}' width='5px' >"
display(HTML('<p>'+image_list+'</p>'))

In [ ]:
from IPython.display import display, Image, clear_output, HTML
import random
import time
import ipywidgets as widgets
from jupyter_ui_poll import ui_events

L1 = Image("12-9.png", width = 300)
L2 = Image("16-12.png", width = 300)
L3 = Image("20-15.png", width = 300)
L4 = Image("14-12.png", width = 300)
L5 = Image("21-18.png", width = 300)
L6 = Image("18-16.png", width = 300)
L7 = Image("10-9.png", width = 300)
L8 = Image("20-18.png", width = 300)
R1 = Image("9-12.png", width = 300)
R2 = Image("12-16.png", width = 300)
R3 = Image("15-20.png", width = 300)
R4 = Image("12-14.png", width = 300)
R5 = Image("18-21.png", width = 300)
R6 = Image("16-18.png", width = 300)
R7 = Image("9-10.png", width = 300)
R8 = Image("18-20.png", width = 300)
Left = [L1,L2,L3,L4,L5,L6,L7,L8]
Right = [R1,R2,R3,R4,R5,R6,R7,R8]
pic_list = 4*Left.copy() + 4*Right.copy()

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return



In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from IPython.display import display, Image, clear_output
import random
import time

def send_to_google_form(data_dict, form_url):
    form_id = form_url.split("/e/")[1].split("/")[0]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

def ANS_test(i):
    global score 
    print("State which side had more dots.")
    time.sleep(1)
    display(pic_list[i])
    time.sleep(0.75)
    clear_output(wait=True)
    
    btn1 = widgets.Button(description="Left")
    btn2 = widgets.Button(description="Right")
    btn1.on_click(register_event) 
    btn2.on_click(register_event)

    print("State which side had more dots.")
    
    panel = widgets.HBox([btn1, btn2])
    display(panel)

    result = wait_for_event(timeout=3)
    clear_output()
    correct_ans = {L1:'Left',L2:'Left',L3:'Left',L4:'Left',L5:'Left',L6:'Left',L7:'Left',L8:'Left',
                  R1:'Right',R2:'Right',R3:'Right',R4:'Right',R5:'Right',R6:'Right',R7:'Right',R8:'Right'}

    if result['description'] == correct_ans[(pic_list[i])]:
        print("correct")
        score = score + 1
    else:
        print("wrong")
        

    time.sleep(1.5)
    clear_output()

    return total_time, score



data_consent_info = """DATA CONSENT INFORMATION:

Please read:
we wish to record your response data
to an anonymised public data repository.
Your data will be used for educational teaching purposes
practising data analysis and visualisation.
Please type yes in the box below if you consent to the upload."""
print(data_consent_info)
result = input("> ")

if result == "yes":
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")

else:

# end code execution by raising an exception

    raise(Exception("User did not consent to continue test."))


id_instructions = """
Enter your anonymised ID
To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress
e.g. if your friend was called Charlie Brown and film star was Tom Cruise
then your unique identifer would be CBTC
"""
print(id_instructions)
user_id = input("> ")
print("User entered id:", user_id)
clear_output()

#test
global score
score = 0
total_time = 0

print("Please enter your name:")
ans1 = input("> ")

print("Please enter your age:")
ans2 = input("> ")

print("Please enter your gender(F/M):")
ans3 = input("> ")
clear_output()

random.shuffle(pic_list)
print("Welcome to the ANS test")
time.sleep(1)
start_time = time.time()
for i in range(64):
    ANS_test(i)
end_time = time.time()
time_taken = end_time - start_time
global total_time
total_time += time_taken

score_percentage = score/64
time_p = "{:.2}".format(total_time)
print("Total time:", time_p)
print("Total score:", score)
score_p = "{:.2%}".format(score_percentage)
print("Score in percentage:",score_p)


data_dict = {
    'User ID': user_id,
    'Name': ans1,
    'Age': ans2,
    'Gender': ans3,
    'Score': score,
    'Total time': total_time,
    'Score in percentage':score_p
}


form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSdB1FqYGsUj55hSj4U55eY_DEI3Qa9zwbcv0RMbmf0dxgYY_Q/viewform'

success = send_to_google_form(data_dict, form_url)


if success:
    print('Data uploaded successfully to Google Forms.')
else:
    print('Failed to upload data to Google Forms.')
